In [15]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# โหลดไฟล์
df = pd.read_csv('material_100hz_data.csv')

# การจัดเรียงตัวแปรแบบอัตโนมัติ (ใช้ตำแหน่งคอลัมน์)
X = df.iloc[:, :-1]  # เลือกทุกคอลัมน์ "ยกเว้น" คอลัมน์สุดท้าย (Features)
y = df.iloc[:, -1]   # เลือกเฉพาะ "คอลัมน์สุดท้าย" (Target/Label)

In [ ]:
# แบ่งข้อมูลเป็น 80% สำหรับเทรน และ 20% สำหรับทดสอบ
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)

# สร้าง XGBoost Regressor
xgb_model = XGBRegressor(random_state=1)
xgb_model.fit(X_train, y_train)

# ทำนาย
y_pred = xgb_model.predict(X_test)

# แสดงผลลัพธ์
print("R2 Score: ", r2_score(y_test, y_pred))
print("MSE: ", mean_squared_error(y_test, y_pred))
print("RMSE: ", mean_squared_error(y_test, y_pred))

R2 Score:  0.8997317151054836
MSE:  3.208815475941383e-05
RMSE:  3.208815475941383e-05


In [1]:
import numpy as np
from scipy.optimize import minimize
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# โหลดข้อมูล
df = pd.read_csv('material_100hz_data.csv')
X = df.iloc[:, :4]   # Proportion1, Proportion2, Temp_C, Pressure_bar
y = df.iloc[:, 4:]   # Alpha_1Hz ถึง Alpha_100Hz

# เทรน XGBoost model สำหรับแต่ละ Hz
models = {}
for col in y.columns:
    model = XGBRegressor(random_state=1, n_estimators=100)
    model.fit(X, y[col])
    models[col] = model

# กำหนดค่า output ที่ต้องการ (ตัวอย่าง: ลดลงจาก 0.5 ไป 0.3)
# คุณสามารถแก้ไขตรงนี้ได้ตามต้องการ
target_output = np.linspace(0.1, 0.98, 100)  # 0.5 ที่ 1Hz ลดลงเรื่อยๆ ถึง 0.3 ที่ 100Hz

# หรือถ้าต้องการกำหนดเอง:
# target_output = [0.5, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4, 0.4, 0.3, ...] # ใส่ครบ 100 ค่า

# ฟังก์ชัน objective: หา input ที่ให้ output ใกล้เคียง target มากที่สุด
def objective(x):
    predictions = []
    for i, col in enumerate(y.columns):
        pred = models[col].predict([x])[0]
        predictions.append(pred)
    predictions = np.array(predictions)
    # คำนวณ Mean Squared Error
    mse = np.mean((predictions - target_output) ** 2)
    return mse

# กำหนดขอบเขตของ input (min, max จากข้อมูลจริง)
bounds = [
    (X['Proportion1'].min(), X['Proportion1'].max()),   # Proportion1
    (X['Proportion2'].min(), X['Proportion2'].max()),   # Proportion2  
    (X['Temp_C'].min(), X['Temp_C'].max()),             # Temp_C
    (X['Pressure_bar'].min(), X['Pressure_bar'].max()), # Pressure_bar
]

# เริ่มต้นจากค่าเฉลี่ย
x0 = X.mean().values

# หาค่า optimal
result = minimize(objective, x0, method='L-BFGS-B', bounds=bounds)

# แสดงผลลัพธ์
print("=" * 50)
print("ค่า Input ที่เหมาะสม:")
print(f"  Proportion1:   {result.x[0]:.4f}")
print(f"  Proportion2:   {result.x[1]:.4f}")
print(f"  Temp_C:        {result.x[2]:.4f}")
print(f"  Pressure_bar:  {result.x[3]:.4f}")
print("=" * 50)
print(f"MSE (ความคลาดเคลื่อน): {result.fun:.6f}")

KeyboardInterrupt: 

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, differential_evolution
from xgboost import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern, WhiteKernel
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 1. โหลดและเตรียมข้อมูล
# ============================================================
print("=" * 60)
print("🔄 กำลังโหลดข้อมูล...")
df = pd.read_csv('experiment_design_27points.csv')

# แยก Input และ Output
X = df[['Proportion1', 'Proportion2', 'Temp_C', 'Pressure_bar']]
y = df.filter(like='Alpha_')  # Alpha_1Hz ถึง Alpha_1000Hz

print(f"✅ จำนวน Data Points: {len(df)}")
print(f"✅ จำนวน Input Features: {X.shape[1]}")
print(f"✅ จำนวน Output Features (Hz): {y.shape[1]}")

# ============================================================
# 2. Standardize Input สำหรับ GPR (ช่วยให้ GPR ทำงานดีขึ้น)
# ============================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ============================================================
# 3. เลือกความถี่ที่ต้องการ (ลดจำนวนเพื่อประหยัดเวลา+หน่วยความจำ)
# ============================================================
# เลือกทุกๆ 10 Hz (1, 10, 20, ..., 100 Hz) หรือปรับได้ตามต้องการ
selected_hz = [f'Alpha_{i}Hz' for i in range(1, 101, 1)]  # 1-100 Hz
selected_hz = [col for col in selected_hz if col in y.columns]
y_selected = y[selected_hz]

print(f"✅ เลือกใช้ความถี่: {len(selected_hz)} ความถี่")

# ============================================================
# 4. เทรนโมเดล XGBoost (ใช้ Cross-Validation เพื่อลด Overfitting)
# ============================================================
print("\n" + "=" * 60)
print("🤖 กำลังเทรน XGBoost Models...")

xgb_models = {}
xgb_scores = []

for col in y_selected.columns:
    # ใช้ regularization parameters เพื่อลด overfitting
    model = XGBRegressor(
        random_state=42,
        n_estimators=50,       # ลดจำนวน trees เพื่อลด overfitting
        max_depth=3,           # ความลึกต่ำ = complexity ต่ำ
        learning_rate=0.1,
        reg_alpha=0.1,         # L1 regularization
        reg_lambda=1.0,        # L2 regularization
        min_child_weight=3,    # ต้องมีข้อมูลขั้นต่ำใน leaf
        subsample=0.8,         # ใช้ข้อมูลบางส่วนในแต่ละ tree
        colsample_bytree=0.8,
        verbosity=0
    )
    
    # Leave-One-Out Cross-Validation (เหมาะกับข้อมูลน้อย)
    loo = LeaveOneOut()
    cv_scores = cross_val_score(model, X, y_selected[col], cv=5, scoring='r2')
    
    # เทรนโมเดลสุดท้าย
    model.fit(X, y_selected[col])
    xgb_models[col] = model
    xgb_scores.append(np.mean(cv_scores))

print(f"✅ XGBoost Average R² (CV): {np.mean(xgb_scores):.4f}")

# ============================================================
# 5. เทรนโมเดล Gaussian Process Regression 
#    (ให้ Uncertainty estimate + interpolation ดี)
# ============================================================
print("\n" + "=" * 60)
print("🧠 กำลังเทรน Gaussian Process Regression Models...")

gpr_models = {}
gpr_scores = []

# Kernel: RBF + White noise (สำหรับข้อมูลน้อย)
kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)

for i, col in enumerate(y_selected.columns):
    gpr = GaussianProcessRegressor(
        kernel=kernel,
        n_restarts_optimizer=5,  # หา hyperparameter ที่ดี
        alpha=0.1,               # noise level เพิ่มความ robust
        normalize_y=True,        # normalize output
        random_state=42
    )
    
    # ใช้ scaled input สำหรับ GPR
    gpr.fit(X_scaled, y_selected[col])
    gpr_models[col] = gpr
    
    # คำนวณ score
    y_pred = gpr.predict(X_scaled)
    score = r2_score(y_selected[col], y_pred)
    gpr_scores.append(score)
    
    if (i + 1) % 20 == 0:
        print(f"  ... เทรนแล้ว {i + 1}/{len(y_selected.columns)} ความถี่")

print(f"✅ GPR Average R² (Training): {np.mean(gpr_scores):.4f}")

# ============================================================
# 6. Ensemble Prediction (รวม XGBoost + GPR)
# ============================================================
def ensemble_predict(x_input, return_std=False):
    """
    ทำนายโดยรวม XGBoost และ GPR
    - XGBoost: ดีสำหรับ non-linear relationships
    - GPR: ให้ uncertainty estimate + smooth interpolation
    """
    x_array = np.array(x_input).reshape(1, -1)
    x_scaled = scaler.transform(x_array)
    
    predictions = []
    uncertainties = []
    
    for col in y_selected.columns:
        # XGBoost prediction
        xgb_pred = xgb_models[col].predict(x_array)[0]
        
        # GPR prediction with uncertainty
        gpr_pred, gpr_std = gpr_models[col].predict(x_scaled, return_std=True)
        gpr_pred = gpr_pred[0]
        gpr_std = gpr_std[0]
        
        # Weighted average: ให้น้ำหนัก GPR มากขึ้นถ้า uncertainty ต่ำ
        weight_gpr = 1 / (1 + gpr_std)  # uncertainty ต่ำ = weight สูง
        weight_xgb = 1 - weight_gpr * 0.5
        
        # Normalize weights
        total_weight = weight_xgb + weight_gpr
        ensemble_pred = (weight_xgb * xgb_pred + weight_gpr * gpr_pred) / total_weight
        
        predictions.append(ensemble_pred)
        uncertainties.append(gpr_std)
    
    if return_std:
        return np.array(predictions), np.array(uncertainties)
    return np.array(predictions)

# ============================================================
# 7. กำหนด Target Output ที่ต้องการ
# ============================================================
print("\n" + "=" * 60)
print("🎯 กำหนด Target Output...")

# ตัวอย่าง: ต้องการค่า Alpha สูง (0.7-0.98) ในช่วง 1-100 Hz
# แก้ไขค่าตรงนี้ตามต้องการ
target_output = np.linspace(0.7, 0.98, len(selected_hz))

# หรือกำหนดเอง:
# target_output = np.array([0.8] * len(selected_hz))  # ต้องการค่าคงที่ 0.8

print(f"  Target range: {target_output.min():.2f} - {target_output.max():.2f}")

# ============================================================
# 8. Objective Function (รวม MSE + Uncertainty penalty)
# ============================================================
def objective_with_uncertainty(x):
    """
    Objective function ที่พิจารณาทั้ง MSE และ Uncertainty
    - ลด MSE: ทำนายใกล้เคียง target
    - ลด Uncertainty: เลือกจุดที่โมเดลมั่นใจ
    """
    predictions, uncertainties = ensemble_predict(x, return_std=True)
    
    # Mean Squared Error
    mse = np.mean((predictions - target_output) ** 2)
    
    # Uncertainty penalty (หลีกเลี่ยงการ extrapolate)
    uncertainty_penalty = np.mean(uncertainties) * 0.1
    
    return mse + uncertainty_penalty

def objective_mse_only(x):
    """Objective แบบ MSE เท่านั้น"""
    predictions = ensemble_predict(x, return_std=False)
    mse = np.mean((predictions - target_output) ** 2)
    return mse

# ============================================================
# 9. กำหนด Bounds และ Constraints
# ============================================================
bounds = [
    (X['Proportion1'].min(), X['Proportion1'].max()),
    (X['Proportion2'].min(), X['Proportion2'].max()),
    (X['Temp_C'].min(), X['Temp_C'].max()),
    (X['Pressure_bar'].min(), X['Pressure_bar'].max()),
]

print(f"\n📊 ขอบเขตของ Input:")
print(f"  Proportion1:  [{bounds[0][0]:.1f}, {bounds[0][1]:.1f}]")
print(f"  Proportion2:  [{bounds[1][0]:.1f}, {bounds[1][1]:.1f}]")
print(f"  Temp_C:       [{bounds[2][0]:.1f}, {bounds[2][1]:.1f}]")
print(f"  Pressure_bar: [{bounds[3][0]:.2f}, {bounds[3][1]:.2f}]")

# ============================================================
# 10. Global Optimization ด้วย Differential Evolution
#     (หลีกเลี่ยง Local Minima)
# ============================================================
print("\n" + "=" * 60)
print("🔍 กำลังหาค่า Optimal ด้วย Differential Evolution...")

result_de = differential_evolution(
    objective_with_uncertainty,
    bounds,
    seed=42,
    maxiter=200,
    popsize=15,
    mutation=(0.5, 1),
    recombination=0.7,
    polish=True,  # polish with L-BFGS-B
    disp=False
)

print(f"✅ DE Optimization สำเร็จ!")

# ============================================================
# 11. Local Refinement ด้วย L-BFGS-B
# ============================================================
print("\n🔧 กำลัง Refine ด้วย L-BFGS-B...")

result_final = minimize(
    objective_mse_only,
    result_de.x,
    method='L-BFGS-B',
    bounds=bounds,
    options={'maxiter': 500}
)

# ============================================================
# 12. แสดงผลลัพธ์
# ============================================================
print("\n" + "=" * 60)
print("🏆 ผลลัพธ์ค่า Input ที่เหมาะสม:")
print("=" * 60)
print(f"  Proportion1:   {result_final.x[0]:.4f}")
print(f"  Proportion2:   {result_final.x[1]:.4f}")
print(f"  Temp_C:        {result_final.x[2]:.4f}")
print(f"  Pressure_bar:  {result_final.x[3]:.4f}")
print("=" * 60)

# แสดง Predicted Output
final_pred, final_std = ensemble_predict(result_final.x, return_std=True)
final_mse = np.mean((final_pred - target_output) ** 2)
final_rmse = np.sqrt(final_mse)

print(f"\n📈 ผลการทำนาย:")
print(f"  MSE:  {final_mse:.6f}")
print(f"  RMSE: {final_rmse:.6f}")
print(f"  Mean Uncertainty: {np.mean(final_std):.4f}")

# แสดงตัวอย่างผลลัพธ์
print(f"\n📊 ตัวอย่างผลทำนาย (ทุกๆ 10 Hz):")
print(f"{'Hz':>6} | {'Target':>8} | {'Predicted':>10} | {'Error':>8} | {'Uncertainty':>12}")
print("-" * 55)
for i in range(0, len(selected_hz), 10):
    hz_label = selected_hz[i].replace('Alpha_', '').replace('Hz', '')
    print(f"{hz_label:>6} | {target_output[i]:>8.4f} | {final_pred[i]:>10.4f} | {abs(final_pred[i]-target_output[i]):>8.4f} | {final_std[i]:>12.4f}")

# ============================================================
# 13. ตรวจสอบว่าอยู่ในช่วง Interpolation หรือ Extrapolation
# ============================================================
print("\n" + "=" * 60)
print("⚠️ ตรวจสอบช่วงข้อมูล (Interpolation Check):")

optimal_x = result_final.x
in_range = True
for i, col in enumerate(['Proportion1', 'Proportion2', 'Temp_C', 'Pressure_bar']):
    min_val = X[col].min()
    max_val = X[col].max()
    val = optimal_x[i]
    
    if val < min_val or val > max_val:
        print(f"  ⚠️ {col}: {val:.4f} อยู่นอกช่วง [{min_val:.2f}, {max_val:.2f}]")
        in_range = False
    else:
        percent = (val - min_val) / (max_val - min_val) * 100
        print(f"  ✅ {col}: {val:.4f} (อยู่ที่ {percent:.1f}% ของช่วง)")

if in_range:
    print("\n✅ ค่าที่ได้อยู่ในช่วง Interpolation - ผลลัพธ์น่าเชื่อถือ!")
else:
    print("\n⚠️ บางค่าอยู่นอกช่วง - ควรทำการทดลองเพิ่มเติมเพื่อยืนยัน")

# ============================================================
# 14. หา Top 5 Candidates (หลายคำตอบที่ดี)
# ============================================================
print("\n" + "=" * 60)
print("🔄 กำลังหาคำตอบที่ดีหลายตัว (Multi-start Optimization)...")

candidates = []
for i in range(10):
    # Random starting point
    x0_random = [
        np.random.uniform(bounds[0][0], bounds[0][1]),
        np.random.uniform(bounds[1][0], bounds[1][1]),
        np.random.uniform(bounds[2][0], bounds[2][1]),
        np.random.uniform(bounds[3][0], bounds[3][1]),
    ]
    
    result_temp = minimize(
        objective_mse_only,
        x0_random,
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 200}
    )
    
    candidates.append({
        'x': result_temp.x,
        'mse': result_temp.fun
    })

# Sort by MSE
candidates = sorted(candidates, key=lambda c: c['mse'])

print("\n🏅 Top 5 คำตอบที่ดี:")
print(f"{'Rank':>4} | {'Proportion1':>11} | {'Proportion2':>11} | {'Temp_C':>8} | {'Pressure':>10} | {'MSE':>10}")
print("-" * 75)
for i, c in enumerate(candidates[:5]):
    print(f"{i+1:>4} | {c['x'][0]:>11.4f} | {c['x'][1]:>11.4f} | {c['x'][2]:>8.4f} | {c['x'][3]:>10.4f} | {c['mse']:>10.6f}")

print("\n" + "=" * 60)
print("✅ เสร็จสิ้น!")

🔄 กำลังโหลดข้อมูล...
✅ จำนวน Data Points: 27
✅ จำนวน Input Features: 4
✅ จำนวน Output Features (Hz): 1000
✅ เลือกใช้ความถี่: 100 ความถี่

🤖 กำลังเทรน XGBoost Models...
✅ XGBoost Average R² (CV): -1.1768

🧠 กำลังเทรน Gaussian Process Regression Models...
  ... เทรนแล้ว 20/100 ความถี่
  ... เทรนแล้ว 40/100 ความถี่
  ... เทรนแล้ว 60/100 ความถี่
  ... เทรนแล้ว 80/100 ความถี่
  ... เทรนแล้ว 100/100 ความถี่
✅ GPR Average R² (Training): 0.9282

🎯 กำหนด Target Output...
  Target range: 0.70 - 0.98

📊 ขอบเขตของ Input:
  Proportion1:  [11.0, 88.0]
  Proportion2:  [12.0, 89.0]
  Temp_C:       [120.3, 129.8]
  Pressure_bar: [4.02, 4.98]

🔍 กำลังหาค่า Optimal ด้วย Differential Evolution...
✅ DE Optimization สำเร็จ!

🔧 กำลัง Refine ด้วย L-BFGS-B...

🏆 ผลลัพธ์ค่า Input ที่เหมาะสม:
  Proportion1:   34.2802
  Proportion2:   63.5963
  Temp_C:        125.2444
  Pressure_bar:  4.6914

📈 ผลการทำนาย:
  MSE:  0.085399
  RMSE: 0.292232
  Mean Uncertainty: 0.2389

📊 ตัวอย่างผลทำนาย (ทุกๆ 10 Hz):
    Hz |   Tar

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, differential_evolution
from xgboost import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 1. โหลดข้อมูล
# ============================================================
print("🔄 กำลังโหลดข้อมูล...")
df = pd.read_csv('experiment_design_27points.csv')

X = df[['Proportion1', 'Proportion2', 'Temp_C', 'Pressure_bar']]
y = df.filter(like='Alpha_')

# ⚡ เลือกแค่ 1-100 Hz (ไม่ใช่ 1000)
selected_hz = [f'Alpha_{i}Hz' for i in range(1, 101)]
y_selected = y[selected_hz]

print(f"✅ Data: {len(df)} rows, {len(selected_hz)} outputs")

# Scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ============================================================
# 2. เทรน XGBoost (เร็วมาก)
# ============================================================
print("\n🤖 เทรน XGBoost...")
xgb_models = {}
for col in y_selected.columns:
    model = XGBRegressor(
        n_estimators=30, max_depth=3, learning_rate=0.1,
        reg_alpha=0.1, reg_lambda=1.0, verbosity=0, random_state=42
    )
    model.fit(X, y_selected[col])
    xgb_models[col] = model
print("✅ XGBoost เสร็จ!")

# ============================================================
# 3. เทรน GPR แค่ 10 ความถี่ (สุ่มตัวอย่าง)
# ============================================================
print("\n🧠 เทรน GPR (10 ความถี่ตัวอย่าง)...")

# ⚡ เลือกแค่ 10 ความถี่ (ทุกๆ 10 Hz)
sample_hz = [f'Alpha_{i}Hz' for i in [1, 10, 20, 30, 40, 50, 60, 70, 80, 100]]

gpr_models = {}
kernel = Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)

for col in sample_hz:
    gpr = GaussianProcessRegressor(
        kernel=kernel, n_restarts_optimizer=2, alpha=0.1,
        normalize_y=True, random_state=42
    )
    gpr.fit(X_scaled, y_selected[col])
    gpr_models[col] = gpr
print("✅ GPR เสร็จ!")

# ============================================================
# 4. Prediction Function (ใช้ XGBoost เป็นหลัก)
# ============================================================
def predict_all(x):
    """ทำนายโดยใช้ XGBoost (เร็ว)"""
    x_arr = np.array(x).reshape(1, -1)
    preds = [xgb_models[col].predict(x_arr)[0] for col in y_selected.columns]
    return np.array(preds)

def get_uncertainty(x):
    """ประเมิน uncertainty จาก GPR (10 จุด)"""
    x_scaled = scaler.transform(np.array(x).reshape(1, -1))
    stds = [gpr_models[col].predict(x_scaled, return_std=True)[1][0] for col in sample_hz]
    return np.mean(stds)

# ============================================================
# 5. Target Output
# ============================================================
target_output = np.linspace(0.7, 0.98, len(selected_hz))
print(f"\n🎯 Target: {target_output[0]:.2f} - {target_output[-1]:.2f}")

# ============================================================
# 6. Objective Function
# ============================================================
def objective(x):
    preds = predict_all(x)
    mse = np.mean((preds - target_output) ** 2)
    return mse

# ============================================================
# 7. Bounds
# ============================================================
bounds = [
    (X['Proportion1'].min(), X['Proportion1'].max()),
    (X['Proportion2'].min(), X['Proportion2'].max()),
    (X['Temp_C'].min(), X['Temp_C'].max()),
    (X['Pressure_bar'].min(), X['Pressure_bar'].max()),
]

# ============================================================
# 8. Optimization (เร็วขึ้น)
# ============================================================
print("\n🔍 กำลัง Optimize (Differential Evolution)...")

result = differential_evolution(
    objective, bounds,
    seed=42, maxiter=50,  # ⚡ ลดจาก 200 เหลือ 50
    popsize=10,           # ⚡ ลดจาก 15 เหลือ 10
    polish=True,
    workers=1,
    disp=True
)

# ============================================================
# 9. ผลลัพธ์
# ============================================================
print("\n" + "=" * 50)
print("🏆 ค่า Input ที่เหมาะสม:")
print("=" * 50)
print(f"  Proportion1:   {result.x[0]:.4f}")
print(f"  Proportion2:   {result.x[1]:.4f}")
print(f"  Temp_C:        {result.x[2]:.4f}")
print(f"  Pressure_bar:  {result.x[3]:.4f}")
print("=" * 50)

final_pred = predict_all(result.x)
mse = np.mean((final_pred - target_output) ** 2)
uncertainty = get_uncertainty(result.x)

print(f"\n📈 MSE: {mse:.6f}")
print(f"📈 RMSE: {np.sqrt(mse):.6f}")
print(f"📈 Uncertainty (GPR): {uncertainty:.4f}")

# ตัวอย่างผล
print(f"\n📊 ตัวอย่างผลทำนาย:")
print(f"{'Hz':>6} | {'Target':>8} | {'Predicted':>10}")
print("-" * 35)
for i in [0, 24, 49, 74, 99]:
    hz = selected_hz[i].replace('Alpha_', '').replace('Hz', '')
    print(f"{hz:>6} | {target_output[i]:>8.4f} | {final_pred[i]:>10.4f}")

print("\n✅ เสร็จสิ้น!")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from scipy.optimize import differential_evolution
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 1. โหลดข้อมูล
# ============================================================
df = pd.read_csv('experiment_design_27points.csv')
X = df[['Proportion1', 'Proportion2', 'Temp_C', 'Pressure_bar']]
y = df.filter(like='Alpha_')

# เลือก 1-100 Hz
selected_hz = [f'Alpha_{i}Hz' for i in range(1, 101)]
y_selected = y[selected_hz]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ============================================================
# 2. เปรียบเทียบโมเดลด้วย Cross-Validation
# ============================================================
print("=" * 60)
print("📊 เปรียบเทียบโมเดลด้วย Leave-One-Out Cross-Validation")
print("=" * 60)

# เลือก 5 ความถี่ทดสอบ
test_cols = ['Alpha_1Hz', 'Alpha_25Hz', 'Alpha_50Hz', 'Alpha_75Hz', 'Alpha_100Hz']

models = {
    'Polynomial (deg=2)': Pipeline([
        ('poly', PolynomialFeatures(degree=2)),
        ('ridge', Ridge(alpha=1.0))
    ]),
    'SVR (RBF)': Pipeline([
        ('scaler', StandardScaler()),
        ('svr', SVR(kernel='rbf', C=10, gamma='scale'))
    ]),
    'Random Forest': RandomForestRegressor(
        n_estimators=50, max_depth=5, random_state=42
    ),
    'XGBoost': XGBRegressor(
        n_estimators=30, max_depth=3, reg_lambda=1, verbosity=0, random_state=42
    ),
}

# ทดสอบแต่ละโมเดล
results = {}
for name, model in models.items():
    scores = []
    for col in test_cols:
        cv_score = cross_val_score(model, X, y_selected[col], cv=5, scoring='r2')
        scores.append(np.mean(cv_score))
    results[name] = np.mean(scores)
    print(f"  {name:25s}: R² = {results[name]:.4f}")

# หาโมเดลที่ดีที่สุด
best_model_name = max(results, key=results.get)
print(f"\n🏆 โมเดลที่ดีที่สุด: {best_model_name} (R² = {results[best_model_name]:.4f})")

# ============================================================
# 3. เทรนโมเดลที่ดีที่สุด
# ============================================================
print(f"\n🔄 เทรน {best_model_name} สำหรับทุกความถี่...")

# สร้างฟังก์ชันสร้างโมเดล
def create_model(name):
    if name == 'Polynomial (deg=2)':
        return Pipeline([
            ('poly', PolynomialFeatures(degree=2)),
            ('ridge', Ridge(alpha=1.0))
        ])
    elif name == 'SVR (RBF)':
        return Pipeline([
            ('scaler', StandardScaler()),
            ('svr', SVR(kernel='rbf', C=10, gamma='scale'))
        ])
    elif name == 'Random Forest':
        return RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42)
    else:  # XGBoost
        return XGBRegressor(n_estimators=30, max_depth=3, reg_lambda=1, verbosity=0, random_state=42)

# เทรนโมเดลสำหรับแต่ละความถี่
trained_models = {}
for col in y_selected.columns:
    model = create_model(best_model_name)
    model.fit(X, y_selected[col])
    trained_models[col] = model

print(f"✅ เทรนเสร็จ {len(trained_models)} โมเดล")

# ============================================================
# 4. Inverse Optimization
# ============================================================
print("\n" + "=" * 60)
print("🔍 หาค่า Input ที่เหมาะสม...")
print("=" * 60)

# Target output
target_output = np.linspace(0.7, 0.98, len(selected_hz))

def predict_all(x):
    x_arr = np.array(x).reshape(1, -1)
    preds = [trained_models[col].predict(x_arr)[0] for col in y_selected.columns]
    return np.array(preds)

def objective(x):
    preds = predict_all(x)
    mse = np.mean((preds - target_output) ** 2)
    return mse

bounds = [
    (X['Proportion1'].min(), X['Proportion1'].max()),
    (X['Proportion2'].min(), X['Proportion2'].max()),
    (X['Temp_C'].min(), X['Temp_C'].max()),
    (X['Pressure_bar'].min(), X['Pressure_bar'].max()),
]

# Optimize
result = differential_evolution(
    objective, bounds,
    seed=42, maxiter=50, popsize=10,
    polish=True, disp=True
)

# ============================================================
# 5. ผลลัพธ์
# ============================================================
print("\n" + "=" * 50)
print("🏆 ค่า Input ที่เหมาะสม:")
print("=" * 50)
print(f"  Proportion1:   {result.x[0]:.4f}")
print(f"  Proportion2:   {result.x[1]:.4f}")
print(f"  Temp_C:        {result.x[2]:.4f}")
print(f"  Pressure_bar:  {result.x[3]:.4f}")
print("=" * 50)

final_pred = predict_all(result.x)
mse = np.mean((final_pred - target_output) ** 2)

print(f"\n📈 MSE: {mse:.6f}")
print(f"📈 RMSE: {np.sqrt(mse):.6f}")

# ตัวอย่างผล
print(f"\n📊 ตัวอย่างผลทำนาย:")
print(f"{'Hz':>6} | {'Target':>8} | {'Predicted':>10} | {'Error':>8}")
print("-" * 45)
for i in [0, 24, 49, 74, 99]:
    hz = i + 1
    err = abs(final_pred[i] - target_output[i])
    print(f"{hz:>6} | {target_output[i]:>8.4f} | {final_pred[i]:>10.4f} | {err:>8.4f}")

print("\n✅ เสร็จสิ้น!")
# ข้อมูล 27 จุด → ใช้ SVR หรือ Polynomial Regression
# ต้องการ Uncertainty → ใช้ GPR (แค่บางความถี่)
# ต้องการความเร็ว → ใช้ XGBoost หรือ Polynomial
# ต้องการความแม่นยำสูงสุด → ลองรันโค้ดด้านบนเพื่อเปรียบเทียบ